<a href="https://colab.research.google.com/github/Socrates-Simao/Primeiro-Projeto---An-lise-Simples/blob/main/An%C3%A1lise_de_compras_de_clientes_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Análise de compras

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
dados = pd.read_json('/content/dados_compras.json')
dados.shape

(780, 6)

In [3]:
dados.head(10)

,Login,Idade,Sexo,Item ID,Nome do Item,Valor
0,Aelalis34,38,Masculino,164,Bone Crushing Silver Skewer,3.37
1,Eolo46,21,Masculino,119,"Stormbringer, Dark Blade of Ending Misery",2.32
2,Assastnya25,34,Masculino,174,Primitive Blade,2.46
3,Pheusrical25,21,Masculino,92,Final Critic,1.36
4,Aela59,23,Masculino,63,Stormfury Mace,1.27
5,Tanimnya91,20,Masculino,10,Sleepwalker,1.73
6,Undjaskla97,20,Masculino,153,Mercenary Sabre,4.57
7,Iathenudil29,29,Feminino,169,"Interrogator, Blood Blade of the Queen",3.32
8,Sondenasta63,25,Masculino,118,"Ghost Reaver, Longsword of Magic",2.77
9,Hilaerin92,31,Masculino,99,"Expiration, Warscythe Of Lost Worlds",4.53


In [4]:
compradores = dados.loc[:, ["Sexo", "Login", "Idade"]]
compradores.head(100)

,Sexo,Login,Idade
0,Masculino,Aelalis34,38
1,Masculino,Eolo46,21
2,Masculino,Assastnya25,34
3,Masculino,Pheusrical25,21
4,Masculino,Aela59,23
...,...,...,...
95,Masculino,Jiskjask80,24
96,Feminino,Yasurra52,22
97,Feminino,Assassasta79,17
98,Masculino,Lamyon68,25


In [5]:
#Vamos remover os itens duplicados.
compradores = compradores.drop_duplicates()
quantos_compradores = compradores.count()[0]
quantos_compradores

573

In [6]:
# Converter saída para DF para uso posterior em análise
pd.DataFrame({"Total de Compradores" : [quantos_compradores]})

,Total de Compradores
0,573


In [7]:
# Cálculos básicos
media_preco_item = dados["Valor"].mean()
total_preco_item = dados["Valor"].sum()
total_item = dados["Valor"].count()
id_item = len(dados["Item ID"].unique())

# Dataframe para os resultados
resumo_calculos = pd.DataFrame({"Número de Itens Únicos" : id_item,
                                     "Número de Compras" : total_item,
                                     "Total de Vendas" : total_preco_item,
                                     "Preço Médio" : [media_preco_item]})

# Data Munging
resumo_calculos = resumo_calculos.round(2)
resumo_calculos ["Preço Médio"] = resumo_calculos["Preço Médio"].map("${:,.2f}".format)
resumo_calculos ["Total de Vendas"] = resumo_calculos["Total de Vendas"].map("${:,.2f}".format)
resumo_calculos = resumo_calculos.loc[:, ["Número de Itens Únicos", "Preço Médio", "Número de Compras", "Total de Vendas"]]

resumo_calculos

,Número de Itens Únicos,Preço Médio,Número de Compras,Total de Vendas
0,183,$2.93,780,"$2,286.33"


# **No código abaixo vamos  gerar um calculo para verificar qual o genero dos compradores. **
#Pode parecer simples, porém temos que ter em mente que o foco é identificador compradores, e sabendo quem gastou mais, a loja pode focar em ofecerer outro tipo de produto para atrair o publico que gastou menos.

In [8]:
# Cálculos básicos
quantidade_genero = compradores["Sexo"].value_counts()
percentual_genero = (quantidade_genero / quantos_compradores) * 100

# Vamor aproveitar para calcular o percentual do genero
generos = pd.DataFrame({"Sexo" : quantidade_genero,
                                    "%" : percentual_genero})

# Data Munging
generos = generos.round(2)
generos ["%"] = generos["%"].map("{:,.1f}%".format)

In [9]:
quantidade_genero

Masculino                465
Feminino                 100
Outro / Não Divulgado      8
Name: Sexo, dtype: int64

In [10]:
percentual_genero

Masculino                81.151832
Feminino                 17.452007
Outro / Não Divulgado     1.396161
Name: Sexo, dtype: float64

In [11]:
# Agrupamentos
total_preco_itens_por_genero = dados.groupby(["Sexo"]).sum()["Valor"].rename("Total de Vendas")
media_preco_itens_por_genero = dados.groupby(["Sexo"]).mean()["Valor"].rename("Average Price")
contagem_compra = dados.groupby(["Sexo"]).count()["Valor"].rename("Número de Compras")
total = total_preco_itens_por_genero / generos["Sexo"]

# Armazenando o resultado em um Dataframe
analise_compra_por_generos = pd.DataFrame({"Número de Compras" : contagem_compra,
                                           "Valor Médio Por Item" : media_preco_itens_por_genero,
                                           "Total de Vendas" : total_preco_itens_por_genero,
                                           "Total Normalizado" : total})

# Manipulação de Dados
analise_compra_por_generos = analise_compra_por_generos.round(2)
analise_compra_por_generos ["Valor Médio Por Item"] = analise_compra_por_generos["Valor Médio Por Item"].map("${:,.2f}".format)
analise_compra_por_generos ["Total de Vendas"] = analise_compra_por_generos["Total de Vendas"].map("${:,.2f}".format)
analise_compra_por_generos ["Total Normalizado"] = analise_compra_por_generos["Total Normalizado"].map("${:,.2f}".format)

<ipython-input-11-e3eb241fc3bf>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_preco_itens_por_genero = dados.groupby(["Sexo"]).sum()["Valor"].rename("Total de Vendas")
<ipython-input-11-e3eb241fc3bf>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  media_preco_itens_por_genero = dados.groupby(["Sexo"]).mean()["Valor"].rename("Average Price")


In [12]:
total_preco_itens_por_genero

Sexo
Feminino                  382.91
Masculino                1867.68
Outro / Não Divulgado      35.74
Name: Total de Vendas, dtype: float64

In [13]:
media_preco_itens_por_genero

Sexo
Feminino                 2.815515
Masculino                2.950521
Outro / Não Divulgado    3.249091
Name: Average Price, dtype: float64

In [14]:
analise_compra_por_generos

,Número de Compras,Valor Médio Por Item,Total de Vendas,Total Normalizado
Feminino,136,$2.82,$382.91,$3.83
Masculino,633,$2.95,"$1,867.68",$4.02
Outro / Não Divulgado,11,$3.25,$35.74,$4.47


In [15]:
fig = px.bar(analise_compra_por_generos, y='Número de Compras', x=analise_compra_por_generos.index)
fig.show()

In [16]:
total

Feminino                 3.829100
Masculino                4.016516
Outro / Não Divulgado    4.467500
dtype: float64

In [17]:
# Cálculos básicos
idade_bins = [0, 9.99, 14.99, 19.99, 24.99, 29.99, 34.99, 39.99, 999]
idade_categoria = ["Menos de 10", "10 a 14", "15 a 19", "20 a 24", "25 a 29", "30 a 34", "35 a 39", "Mais de 40"]

dados["Range de Idades"] = pd.cut(dados["Idade"], idade_bins, labels=idade_categoria)

# Cálculos básicos
contagem_idade = dados["Range de Idades"].value_counts()
media_preco_item_por_idade = dados.groupby(["Range de Idades"]).mean()["Valor"]
total_preco_item_por_idade = dados.groupby(["Range de Idades"]).sum()["Valor"]
percentual_por_idade = (contagem_idade / quantos_compradores) * 100

# Dataframe para os resultados
idade = pd.DataFrame({"Contagem": contagem_idade, "%": percentual_por_idade, "Valor Unitario": media_preco_item_por_idade, "Valor Total de Compra": total_preco_item_por_idade})

# Manipulação de Dados
idade ["Valor Unitario"] = idade["Valor Unitario"].map("${:,.2f}".format)
idade ["Valor Total de Compra"] = idade["Valor Total de Compra"].map("${:,.2f}".format)
idade ["%"] = idade["%"].map("{:,.2f}%".format)

<ipython-input-17-53fe65aa95a5>:9: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-17-53fe65aa95a5>:10: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [18]:
idade = idade.sort_index()
idade

,Contagem,%,Valor Unitario,Valor Total de Compra
Menos de 10,28,4.89%,$2.98,$83.46
10 a 14,35,6.11%,$2.77,$96.95
15 a 19,133,23.21%,$2.91,$386.42
20 a 24,336,58.64%,$2.91,$978.77
25 a 29,125,21.82%,$2.96,$370.33
30 a 34,64,11.17%,$3.08,$197.25
35 a 39,42,7.33%,$2.84,$119.40
Mais de 40,17,2.97%,$3.16,$53.75


In [19]:
# Cálculos básicos
total_compras = dados.groupby(["Login"]).sum()["Valor"].rename("Valor Total de Compra")
media_compras = dados.groupby(["Login"]).mean()["Valor"].rename("Valor Médio de Compra")
numero_compras = dados.groupby(["Login"]).count()["Valor"].rename("Número de Compras")

# Dataframe para os resultados
dados_usuarios = pd.DataFrame({"Valor Total de Compra": total_compras, "Valor Médio de Compra": media_compras, "Número de Compras": numero_compras})

# Manipulação de Dados
dados_usuarios ["Valor Total de Compra"] = dados_usuarios["Valor Total de Compra"].map("${:,.2f}".format)
dados_usuarios ["Valor Médio de Compra"] = dados_usuarios["Valor Médio de Compra"].map("${:,.2f}".format)
dados_usuarios.sort_values("Valor Total de Compra", ascending=False).head(10)

<ipython-input-19-292d6381a3db>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-19-292d6381a3db>:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Valor Total de Compra,Valor Médio de Compra,Número de Compras
Login,,,
Qarwen67,$9.97,$2.49,4
Sondim43,$9.38,$3.13,3
Tillyrin30,$9.19,$3.06,3
Lisistaya47,$9.19,$3.06,3
Tyisriphos58,$9.18,$4.59,2
Frichaya88,$9.16,$3.05,3
Assassasta79,$9.12,$4.56,2
Lisossan98,$9.10,$4.55,2
Yadanun74,$9.09,$3.03,3


In [20]:
# Cálculos básicos
total_compras = dados.groupby(["Nome do Item"]).sum()["Valor"].rename("Valor Total de Compra")
media_compras = dados.groupby(["Nome do Item"]).mean()["Valor"].rename("Valor Médio de Compra")
numero_compras = dados.groupby(["Nome do Item"]).count()["Valor"].rename("Número de Compras")

# Dataframe para os resultados
dados_usuarios = pd.DataFrame({"Valor Total de Compra": total_compras, "Valor Médio de Compra": media_compras, "Número de Compras": numero_compras})

# Manipulação de Dados
dados_usuarios ["Valor Total de Compra"] = dados_usuarios["Valor Total de Compra"].map("${:,.2f}".format)
dados_usuarios ["Valor Médio de Compra"] = dados_usuarios["Valor Médio de Compra"].map("${:,.2f}".format)
dados_usuarios.sort_values("Número de Compras", ascending=False).head(5)

<ipython-input-20-5dda4fe1918a>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-20-5dda4fe1918a>:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Valor Total de Compra,Valor Médio de Compra,Número de Compras
Nome do Item,,,
Final Critic,$38.60,$2.76,14
Arcane Gem,$24.53,$2.23,11
"Betrayal, Whisper of Grieving Widows",$25.85,$2.35,11
Stormcaller,$34.65,$3.47,10
Woeful Adamantite Claymore,$11.16,$1.24,9


In [21]:
# Cálculos básicos
total_compras = dados.groupby(["Nome do Item"]).sum()["Valor"].rename("Valor Total de Compra")
media_compras = dados.groupby(["Nome do Item"]).mean()["Valor"].rename("Valor Médio de Compra")
numero_compras = dados.groupby(["Nome do Item"]).count()["Valor"].rename("Número de Compras")

# Dataframe para os resultados
dados_usuarios = pd.DataFrame({"Valor Total de Compra": total_compras, "Valor Médio de Compra": media_compras, \
                          "Número de Compras": numero_compras})

# Manipulação de Dados
dados_usuarios ["Valor Total Compra"] = dados_usuarios["Valor Total de Compra"]
dados_usuarios ["Valor Total de Compra"] = dados_usuarios["Valor Total de Compra"].map("${:,.2f}".format)
dados_usuarios ["Valor Médio de Compra"] = dados_usuarios["Valor Médio de Compra"].map("${:,.2f}".format)


display(dados_usuarios.sort_values("Valor Total Compra", ascending=False).head(5)[ \
    ['Valor Total de Compra','Valor Médio de Compra','Número de Compras']])

<ipython-input-21-71d30322d693>:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-21-71d30322d693>:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Valor Total de Compra,Valor Médio de Compra,Número de Compras
Nome do Item,,,
Final Critic,$38.60,$2.76,14
Retribution Axe,$37.26,$4.14,9
Stormcaller,$34.65,$3.47,10
Spectral Diamond Doomblade,$29.75,$4.25,7
Orenmir,$29.70,$4.95,6


In [22]:
fig = px.bar(dados_usuarios, x='Valor Total de Compra', y=dados_usuarios.index)
fig.show()